In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('sentiment_review.csv')
df=df.drop(['Symptoms', 'Rating', 'Unnamed: 0'], axis=1)
df.Drug = df.Drug.str.upper()

In [3]:
predicted_disease = 'pneumonia'

In [4]:
# function to get drugs based on sentiment review
def _DrugRecommender(df=df, predicted_disease=predicted_disease):
    #df['Rating'] = np.where(df['Rating']>7, 1, 0)
    df = df.loc[df['Disease'] == predicted_disease]
    df = df.loc[df['sentiment_review'] == 'positive']
    df = df.sort_values(by = ['UsefulCount'], ascending=False)
    df = df[['Disease', 'Drug']]
    df = df.drop_duplicates()
    df = df.reset_index()
    df = df.drop(['index'], axis=1)
    df = df.head()

    return df

In [5]:
drug = _DrugRecommender()

In [6]:
print('List of recommened drugs based on useful count and sentiment analysis')
drug

List of recommened drugs based on useful count and sentiment analysis


,Disease,Drug
0,pneumonia,LEVOFLOXACIN
1,pneumonia,LEVAQUIN
2,pneumonia,AUGMENTIN
3,pneumonia,AMOXICILLIN / CLAVULANATE
4,pneumonia,DOXYCYCLINE


In [7]:
sd = pd.read_csv('drugLib_processed.csv')
sd = sd.drop(['condition', 'commentsReview'], axis=1)
sd = sd.dropna()
sd.head()

,rating,urlDrugName,effectiveness,sideEffects,benefitsReview,sideEffectsReview
0,1,dispermox,Ineffective,Severe Side Effects,"Amoxicillin did little, if any, good for the i...",I had a severe allergic reaction in the form o...
1,1,lexapro,Marginally Effective,Severe Side Effects,"Initially, Lexapro helped alleviate some of th...","Although some stress was alleviated, I noticed..."
2,1,zocor,Ineffective,Extremely Severe Side Effects,the treatment was to help lower my colesterol ...,"muscle pain, loss of mobility, depresion, head..."
3,1,ventolin-hfa,Ineffective,Mild Side Effects,None,"Increased difficulty breathing, strange taste."
4,1,neurontin,Ineffective,Severe Side Effects,No treatment benefits. the edema was so severe...,"whole body swelling (even my eyelids), palpita..."


In [8]:
sd.sideEffects = sd.sideEffects.replace('Moderate Side Effects', 2)
sd.sideEffects = sd.sideEffects.replace('Mild Side Effects', 2)
sd.sideEffects = sd.sideEffects.replace('Severe Side Effects', 1)
sd.sideEffects = sd.sideEffects.replace('No Side Effects', 3)
sd.sideEffects = sd.sideEffects.replace('Extremely Severe Side Effects', 0)

In [9]:
sd.effectiveness = sd.effectiveness.replace('Considerably Effective', 2)
sd.effectiveness = sd.effectiveness.replace('Highly Effective', 3)
sd.effectiveness = sd.effectiveness.replace('Marginally Effective', 1)
sd.effectiveness = sd.effectiveness.replace('Moderately Effective', 1)
sd.effectiveness = sd.effectiveness.replace('Ineffective', 0)

In [10]:
sd = sd[['urlDrugName', 'rating', 'effectiveness', 'sideEffects', 'sideEffectsReview']]

In [11]:
sd = sd.rename(columns={"urlDrugName": "Drug", "rating": "Rating", "effectiveness": "Effectivness Rating",
                       "sideEffects": "Side Effect Rating", "sideEffectsReview": "Side Effects"})
sd.Drug = sd.Drug.str.upper()

In [12]:
print('Drug with possible side effects')
sd

Drug with possible side effects


,Drug,Rating,Effectivness Rating,Side Effect Rating,Side Effects
0,DISPERMOX,1,0,1,I had a severe allergic reaction in the form o...
1,LEXAPRO,1,1,1,"Although some stress was alleviated, I noticed..."
2,ZOCOR,1,0,0,"muscle pain, loss of mobility, depresion, head..."
3,VENTOLIN-HFA,1,0,2,"Increased difficulty breathing, strange taste."
4,NEURONTIN,1,0,1,"whole body swelling (even my eyelids), palpita..."
...,...,...,...,...,...
4127,PAXIL,10,3,2,During the second week I experienced stomach p...
4128,SANCTURA-XR,10,2,2,Dry mouth if I fast from food. If I eat norma...
4129,TRAZODONE,10,3,3,I have noticed no side effects. I have never n...
4130,TOPAMAX,10,3,2,Mild loss of appetite. Loss of interest in alc...


In [13]:
sd.to_csv("Sideeffects_preprocessed.csv");

In [14]:
def _probScore(df, w1, w2, w3):
    return(1-(df[w1] * df[w2] * df[w3]).sum()/df[w1].sum()/10)

In [15]:
def _getSE(df, sedf, l):
    sdf = sedf
    sdf = sdf.loc[sdf['Drug'] == l]
    #print(sdf)
    w = sdf.groupby(["Drug"]).apply(_probScore, "Rating", "Effectivness Rating", "Side Effect Rating")
    sdf1 = sdf.loc[sdf['Effectivness Rating'] == 0]
    sdf1 = sdf1.loc[sdf['Side Effect Rating'] == 0]
    sdf1 = sdf1.loc[sdf['Rating'] <= 1 ]
    sdf1 = sdf1.reset_index()
    sdf1 = pd.DataFrame(sdf['Side Effects'])
    df1 = pd.DataFrame(w, columns=['Prob. of Side Effect'])
    df1 = df1.reset_index()
    #print(sdf[:1]['Side Effects'])
    df2 = pd.DataFrame(sdf['Side Effects'])
    df2 = df2.reset_index()
    df2 = df2.drop(['index'], axis=1)
    dd = pd.concat([df1, df2], axis=1)
    
    return dd

In [16]:
def _getSideEffects(df, sedf):
    l = list(df.Drug)
    #print(l)
    for i in range(0, 1):
        df1 = _getSE(df, sedf, l[i])
    dd = df1
    for i in range(1, len(l)):
        df2 = _getSE(df, sedf, l[i])
        dd = pd.concat([dd, df2])
    return dd

In [17]:
se = _getSideEffects(drug, sd)

In [18]:
recommender = drug.set_index('Drug').join(se.set_index('Drug'))
recommender = recommender.reset_index()
recommender['Prob. of Side Effect'] = recommender['Prob. of Side Effect'].fillna(0)
recommender['Prob. of Side Effect'] = recommender['Prob. of Side Effect'].fillna('Not Available')
print('Mapped recommended drugs with possible side effects and probabilistic score')
recommender

Mapped recommended drugs with possible side effects and probabilistic score


,Drug,Disease,Prob. of Side Effect,Side Effects
0,AMOXICILLIN / CLAVULANATE,pneumonia,0.000000,NaN
1,AUGMENTIN,pneumonia,0.445517,"Cranky, horrible diaper rash immediately follo..."
2,DOXYCYCLINE,pneumonia,0.492683,Severe abdominal spasms/pain and diarrhea
3,LEVAQUIN,pneumonia,0.482222,"SEVERE shooting, burning pains in all four ext..."
4,LEVOFLOXACIN,pneumonia,0.000000,NaN


In [19]:
#sorting based on probabilistic score
recommender = recommender.sort_values('Prob. of Side Effect')
print("The recommended Drugs for the given Disease is:")
recommender

The recommended Drugs for the given Disease is:


,Drug,Disease,Prob. of Side Effect,Side Effects
0,AMOXICILLIN / CLAVULANATE,pneumonia,0.000000,NaN
4,LEVOFLOXACIN,pneumonia,0.000000,NaN
1,AUGMENTIN,pneumonia,0.445517,"Cranky, horrible diaper rash immediately follo..."
3,LEVAQUIN,pneumonia,0.482222,"SEVERE shooting, burning pains in all four ext..."
2,DOXYCYCLINE,pneumonia,0.492683,Severe abdominal spasms/pain and diarrhea
